In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import *
from functools import *
from learn import em_learn, svd_learn_new
from data import *
plt.rcParams["text.usetex"] = True

import itertools
from data import *
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform, pdist
from scipy.cluster.vq import kmeans2, ClusterError
import time

# import cvxpy as cp
USE_CVXPY = False
CVXPY_SOLVER = "Gurobi"


In [59]:

def svd_learn(sample, n, L=None, verbose=None, stats={}):
    Os = np.moveaxis(sample.all_trail_probs(), 1, 0)

    svds = [np.linalg.svd(Os[j], full_matrices=True) for j in range(n)]

    if verbose:
        for i, (_, s, _) in enumerate(svds):
            print(f"{i}: {s[:L+1]} ...")

    Ps_ = np.zeros((n, L, n))
    Qs_ = np.zeros((n, L, n))
    for j, (u, s, vh) in enumerate(svds):
        Ps_[j, 0 : min(n, L), :] = u[:, 0:L].T
        Qs_[j, 0 : min(n, L), :] = (np.diag(s) @ (vh))[0:L, :]

    A = np.zeros((2 * n * L, n**2))
    for j in range(n):
        A[L * j : L * (j + 1), n * j : n * (j + 1)] = Ps_[j]
        A[L * (n + j) : L * (n + j + 1), j + n * (np.arange(n))] = -Qs_[j]

    _, s, vh = np.linalg.svd(A.T, full_matrices=True)
    small = list(s < 1e-5)
    if True in small:
        fst = small.index(True)
        if verbose:
            print(2 * L * n - fst, L, s[[fst - 1, fst]])
    B = vh[-L:]
    Bre = np.moveaxis(B.reshape((L, L, 2 * n), order="F"), -1, 0)
    Ys_ = Bre[0:n]
    Zs_ = Bre[n : 2 * n]

    Xs = [
        np.linalg.pinv(Zs_[j] @ Ys_[j].T) @ (Zs_[j + 1] @ Ys_[j + 1].T)
        for j in range(n - 1)
    ]
    X = np.sum(Xs, axis=0)
    _, R_ = np.linalg.eig(X)
    d, _, _, _ = np.linalg.lstsq(
        (R_.T @ Ys_[0] @ Ps_[0]).T, Os[0] @ np.ones(n), rcond=None
    )

    R = np.diag(d) @ R_.T
    Ys = R @ Ys_
    Ps = np.array([Y @ P_ for Y, P_ in zip(Ys, Ps_)])
    Ss = np.array([R @ Z_ @ Y_.T @ R.T for Z_, Y_ in zip(Zs_, Ys_)])

    S_ = np.zeros((L, n))
    Ms_ = np.zeros((L, n, n))
    for l in range(L):
        for i in range(n):
            S_[l, i] = Ss[i, l, l]
            for j in range(n):
                Ms_[l, i, j] = Ps[j, l, i] / S_[l, i]

    S_ = np.abs(S_)
    Ms_ = np.abs(Ms_)
    learned_mixture = Mixture(S_, Ms_)
    learned_mixture.normalize()
    return learned_mixture


In [60]:
def disconnect(mixture, l, n_parts=2, start=0, partition=None, direction=None):
    n = mixture.n
    if partition is None:
        partition = np.array_split(range(start, start+n), n_parts)
    for p1, p2 in combinations(partition, 2):
        for part1, part2 in [[p1, p2], [p2, p1]]:
            for i, j in product(part1, part2):
                if direction is None or np.sign(i - j) == direction:
                    mixture.Ms[l, i % n, j % n] = 0
    mixture.normalize()

def disconnect_into(mixture, r):
    mixture = mixture.copy()
    if r is None:
        return mixture
    assert (mixture.L <= r <= mixture.n / 2)
    add_n_cc = round(r) - mixture.L
    n_ccs = np.random.rand(mixture.L)
    n_ccs = (add_n_cc * n_ccs / np.sum(n_ccs)).astype(int)
    n_ccs[np.random.choice(mixture.L, add_n_cc - np.sum(n_ccs), replace=False)] += 1
    n_ccs += 1
    for l, n_cc in enumerate(n_ccs):
        xs = np.array(range(mixture.n // 2))
        np.random.shuffle(xs)
        partition = [[x] for x in xs[:n_cc]]
        for x in xs[n_cc:]:
            i = np.random.choice(n_cc)
            partition[i].append(x)
        partition = [[i for x in part for i in [2 * x, 2 * x + 1]] for part in partition]
        disconnect(mixture, l, partition=partition)
    return mixture

def plot(df, x, y, fill=True, **kwargs):
    plt.xlabel(x)
    plt.ylabel(y)
    for learner, learner_df in df.groupby("learner"):
        grp = learner_df.groupby(x)[y]
        med = grp.median()
        plt.plot(med.index, med, label=f"{learner} ({y})", marker="o", **kwargs)
        if fill: plt.fill_between(med.index, grp.quantile(0.25), grp.quantile(0.75), alpha=0.2)

n_trials = 3

learners = {
    "CA-SVD": svd_learn_new,
    "CA-SVD'": lambda d, n, L: svd_learn_new(d, n, L, sample_dist=0.01),
    "GKV-SVD": svd_learn,
    "EM2": lambda d, n, L: em_learn(d, n, L, max_iter=2),
    "EM5": lambda d, n, L: em_learn(d, n, L, max_iter=5),
    "EM20": lambda d, n, L: em_learn(d, n, L, max_iter=20),
    "EM50": lambda d, n, L: em_learn(d, n, L, max_iter=50),
    "EM100": lambda d, n, L: em_learn(d, n, L, max_iter=100),
    "EM-converge": em_learn,
    "CA-SVD-EM2": lambda d, n, L: svd_learn_new(d, n, L, em_refine_max_iter=2),
    "CA-SVD-EM5": lambda d, n, L: svd_learn_new(d, n, L, em_refine_max_iter=5),
    "CA-SVD-EM20": lambda d, n, L: svd_learn_new(d, n, L, em_refine_max_iter=20),
    "CA-SVD-EM100": lambda d, n, L: svd_learn_new(d, n, L, em_refine_max_iter=100),
}

### Tool functions
def count_3_from_seq(seq, n):
    """
    seq: discretized sequence
    n: number of categories
    """
    all_trail_probs = np.zeros((n, n, n)) 
    for i in range(len(seq) // 3):
        x = seq[3*i:3*(i+1)]
        all_trail_probs[tuple(x)] += 1
        num_visited[x] += 1
    return Distribution.from_all_trail_probs(all_trail_probs / np.sum(all_trail_probs))
    
def count_3_from_trails(trail, n):
    all_trail_probs = np.zeros((n, n, n))
    for x in trail:
        
        all_trail_probs[tuple(x)] += 1
    return Distribution.from_all_trail_probs(all_trail_probs / np.sum(all_trail_probs))

def learn_distribution(learner, distribution, n, L):
    learned_mixture = learners[learner](distribution, n, L)
    return Distribution.from_mixture(learned_mixture, 3)

def learn_mix_from_seq(seq,learner, n, L):
    """
    seq: discretized time series: an 1-d array
    learner: 
    """
    trail_empirical_distribution = count_3_from_seq(seq, n)
    if np.isnan(trail_empirical_distribution.all_trail_probs()).any() or np.isinf(trail_empirical_distribution.all_trail_probs()).any():
        print("Inf or NAN values")
        print(trail_empirical_distribution.all_trail_probs())
        
    return  learners[learner](trail_empirical_distribution, n, L)

def trails_3(xs):
    trails = []
    for i in range(len(xs) // 3):
        x = xs[3*i:3*(i+1)]
        trails.append(x)
    return np.array(trails)

# Question: how to learn from multi-dimension 
def transitions(n, trails):
    n_samples = trails.shape[0]
    c = np.zeros([n_samples, n, n], dtype=int)
    for t, trail in enumerate(trails):
        i = trail[0]
        for j in trail[1:]:
            c[t, i, j] += 1
            i = j
    return c

def likelihood(mixture, trails, counts=None, log=False):
    if counts is None: counts = transitions(mixture.n, trails)
    logS = np.log(mixture.S + 1e-10)
    logTs = np.log(mixture.Ms + 1e-10)

    logl = logS[:, trails[:,0]]
    logl += np.sum(logTs[:, :, :, None] * np.moveaxis(counts, 0, 2)[None, :, :, :], axis=(1,2))
    if log: return logl
    probs = np.exp(logl - np.max(logl, axis=0))
    probs /= np.sum(probs, axis=0)[None, :]
    return probs


In [63]:
# bins: 10 20 
# reading and discretization
num_categories = 60
n = num_categories

all_trail_probs = np.zeros((n, n, n))
num_visited = np.zeros(num_categories)

df = pd.read_csv('energydata_complete.csv')
# consider one column first
#df = df['RH_5']
#xs = pd.cut(df, bins=num_categories, labels=False)
res = pd.qcut(df['RH_5'],n, labels=False, retbins=True, precision=3, duplicates='raise')
# do equal-depth p
xs = np.array(list(res[0]))
bins = res[1]


In [68]:
window = len(xs)//20
L = 5
correct_count = 0
error = []
neg_ll = []
predict = []
for i in range(100):
    subseq = xs[i:i+window]
    learned_mix = learn_mix_from_seq(subseq,'GKV-SVD', num_categories, L)
    chain_prob = likelihood(learned_mix, np.atleast_2d(subseq[-2:]))
    #most_likely_index = np.argmax(chain_prob)
    #multi_dim_index = np.unravel_index(most_likely_index, learned_mix.S.shape)
    # Based on likelihood probability to find the most likely chain.
    most_likely_chain = np.argmax(chain_prob)
    prob_next_step = learned_mix.Ms[most_likely_chain, subseq[window - 1], :]
    neg_log_likelihood = -np.log(prob_next_step[xs[i + window]]) + np.log(np.max(prob_next_step))
    sorted_indices = np.argsort(prob_next_step)
    predict.append(np.argmax(prob_next_step))
    
    rank = np.where(sorted_indices == xs[i + window])[0][0]

    neg_ll.append(neg_log_likelihood)
    error.append(59 - rank)


/var/folders/kx/06y42x8s5571j9jqkpc5wf500000gn/T/ipykernel_15861/4213786882.py:53: RuntimeWarning: invalid value encountered in double_scalars
  Ms_[l, i, j] = Ps[j, l, i] / S_[l, i]
/Users/hayashimiyako/Desktop/DirectedStudy/WWW2023/data.py:180: RuntimeWarning: invalid value encountered in divide
  self.S /= np.sum(self.S)
/var/folders/kx/06y42x8s5571j9jqkpc5wf500000gn/T/ipykernel_15861/4213786882.py:51: ComplexWarning: Casting complex values to real discards the imaginary part
  S_[l, i] = Ss[i, l, l]
/var/folders/kx/06y42x8s5571j9jqkpc5wf500000gn/T/ipykernel_15861/4213786882.py:53: RuntimeWarning: invalid value encountered in cdouble_scalars
  Ms_[l, i, j] = Ps[j, l, i] / S_[l, i]
/var/folders/kx/06y42x8s5571j9jqkpc5wf500000gn/T/ipykernel_15861/4213786882.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  Ms_[l, i, j] = Ps[j, l, i] / S_[l, i]


In [66]:
# mean rank error 
error = np.array(error)
print(np.mean(error))


34.49


In [44]:
def trail_prob_predict(window, L):
    correct_count = 0
    error = []
    neg_ll = []
    for i in range(100):
        subseq = xs[i:i+window]

        learned_mix = learn_mix_from_seq(subseq,'GKV-SVD', num_categories, L)

        chain_prob = likelihood(learned_mix, np.atleast_2d(subseq[-2:]))
        #most_likely_index = np.argmax(chain_prob)
        #multi_dim_index = np.unravel_index(most_likely_index, learned_mix.S.shape)
        # Based on likelihood probability to find the most likely chain.
        
        most_likely_chain = np.argmax(chain_prob)

        prob_next_step = learned_mix.Ms[most_likely_chain, subseq[window - 1], :]
        neg_log_likelihood = -np.log(prob_next_step[xs[i + window]]) + np.log(np.max(prob_next_step))
        sorted_indices = np.argsort(prob_next_step)
        
        rank = np.where(sorted_indices == xs[i + window])[0][0]

        neg_ll.append(neg_log_likelihood)
        error.append(59 - rank)
    error = np.array(error)
    return np.mean(error)

In [45]:
for window in [10 * i for i in range(1,10)]:
    print(trail_prob_predict(window, 5))

/var/folders/kx/06y42x8s5571j9jqkpc5wf500000gn/T/ipykernel_15861/4213786882.py:53: RuntimeWarning: invalid value encountered in double_scalars
  Ms_[l, i, j] = Ps[j, l, i] / S_[l, i]
/Users/hayashimiyako/Desktop/DirectedStudy/WWW2023/data.py:180: RuntimeWarning: invalid value encountered in divide
  self.S /= np.sum(self.S)


KeyboardInterrupt: 

In [48]:
%pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 4.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 4.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [56]:
import statsmodels.api as sm
import pandas as pd


In [57]:

#endog = pd.read_csv('energydata_complete.csv')['RH_5']
endog = pd.DataFrame(xs, columns=['self_generate'])
# We could also fit a more complicated model with seasonal components.
# As an example, here is an SARIMA(1,1,1) x (0,1,1,4):
mod_sarimax = sm.tsa.SARIMAX(endog, order=(1,1,1),
                             seasonal_order=(0,1,1,4))
res_sarimax = mod_sarimax.fit()

res = mod_sarimax.filter(res_sarimax.params)

# Show the summary of results
pred = res.get_prediction().predicted_mean
discretized_pred = np.digitize(pred, bins, right=False)


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.64306D+00    |proj g|=  1.00117D-01

At iterate    5    f=  2.54111D+00    |proj g|=  2.87708D-02

At iterate   10    f=  2.51285D+00    |proj g|=  2.80728D-03

At iterate   15    f=  2.50949D+00    |proj g|=  9.57891D-03

At iterate   20    f=  2.50878D+00    |proj g|=  9.34617D-04

At iterate   25    f=  2.50875D+00    |proj g|=  1.58035D-05

At iterate   30    f=  2.50875D+00    |proj g|=  3.38540D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac

KeyboardInterrupt: 

In [ ]:
# ARIMA

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error



count = 0
ctn_series = df['RH_5'].to_numpy()
for i in range(100):
    subseq = ctn_series[i:i+window]
    model = ARIMA(subseq, order=(5,1,0))  # Example order, adjust based on ACF/PACF analysis or domain knowledge
    model_fit = model.fit()

    # Predict the next point
    predict_ctn = model_fit.forecast()[0]
    predict_dct = np.digitize(predict_ctn, bins, right=False)
    #prediction = np.digitize(, bins, right=False)
    print(predict_dct, xs[i + window])
    if (predict_dct ==  xs[i + window]):
        count+=1
print(count)



# 2024-2-27
def start_prob_predict(window, num_chains):
    window = len(xs)//10
    L = num_chains
    correct_count = 0
    error = []
    for i in range(100):
        subseq = xs[i:i+window]

        learned_mix = learn_mix_from_seq(subseq,'GKV-SVD', num_categories, L)
        #chain_prob = likelihood(learned_mix, trails_3(xs))
        most_likely_index = np.argmax(learned_mix.S)
        multi_dim_index = np.unravel_index(most_likely_index, learned_mix.S.shape)
        # Based on starting probability to find the most likely chain.
        most_likely_chain = np.argmax(learned_mix.S[:,xs[i + window - 1]])

        predict_next_step = np.argmax(learned_mix.Ms[most_likely_chain, xs[i + window - 1], :])
        
        print(predict_next_step, xs[i + window])
        if (xs[i + window] in predict_next_step ):
            correct_count+=1
        error.append(abs(predict_next_step - xs[i + window]))
    return error


In [51]:
print(discretized_pred)
print(np.mean(np.abs(xs - discretized_pred)[1:]))  # mean difference



[0 0 1 ... 0 0 0]
28.13778543252403
